In [18]:
import pandas as pd
import numpy as np

In [19]:
df = pd.read_csv('movie5.csv')

In [1]:
df.isnull().sum()

In [4]:
df.duplicated().sum()

In [22]:
df.drop_duplicates(keep = 'first',inplace = True)

In [24]:
def convert(x):
    temp = x.replace(',','').split()
    return temp

In [25]:
def converter(x):
    temp = x.split(',')
    return temp

In [26]:
df['genre'] = df['genre'].apply(convert)
df['actor'] = df['actor'].apply(converter)
df['director'] = df['director'].apply(converter)
df['desc'] = df['desc'].apply(convert)

In [27]:
df['actor'] = df['actor'].apply(lambda x:[i.replace(' ','') for i in x])
df['director'] = df['director'].apply(lambda x:[i.replace(' ','') for i in x])
df['genre'] = df['genre'].apply(lambda x:[i.replace(' ','') for i in x])

In [29]:
df['tags'] = df['desc'] + df['genre'] + df['actor'] + df['director']

In [30]:
new_df = df[['title','tags','poster','genre','details']]

In [2]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [3]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [35]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y = []
    
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [34]:
new_df['tags'][2]

'an emotional thriller that represents a higher quest a search for lost values of empathy and integrity. drama thriller yamigautam piaabajpai rohitbasfore yogeshbhardwaj aniruddharoychowdhury'

In [5]:
new_df['tags'] = new_df['tags'].apply(stem)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000,stop_words = 'english')

In [38]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [40]:
cv.get_feature_names()

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [45]:
sorted(list(enumerate(similarity[1000])),reverse = True,key = lambda x:x[1])[1:6]

In [43]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse = True,key = lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [6]:
recommend('Selfiee')

In [48]:
import pickle

In [49]:
pickle.dump(new_df.to_dict(),open('movies.pkl','wb'))

In [50]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [52]:
type(new_df['details'].iloc[0])

str